In [88]:
import networkx as nx
import numpy as np
import pickle

import dgl
import torch
import dgl.nn
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [89]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [152]:
path = './processed_data/'
class TDGNN(nn.Module):
    def __init__(self,in_feature,hidden,out_feature):
        super(TDGNN, self).__init__()
        self.fc1 = nn.Linear(in_feature,hidden,bias=True)
        self.fc2 = nn.Linear(hidden,out_feature,bias=True)
        self.dropout = nn.Dropout(0.5)
        
    def reset_param(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.fc1.weight, gain=gain)
        nn.init.xavier_normal_(self.fc2.weight, gain=gain)
            
    def forward(self,feature):
        
        h =self.fc1(feature)
       
        h = self.fc2(h)
        return torch.sigmoid(h)

In [153]:
with open('./ia-contact-features/train_cat_features_v1','rb') as f:
    train_features = pickle.load(f)
with open('./ia-contact-features/test_cat_features_v1','rb') as f:
    test_features = pickle.load(f)

In [193]:
def train(model,train_features):
    opt = torch.optim.Adam(model.parameters(),lr=1e-3)
    loss_fn = torch.nn.MSELoss(reduction='mean')
    for epoch in range(200):
        running_loss = 0.0
        print("epoch %d begins"%epoch)
        model.train()
        for feature,label in train_features:
            feature = feature.float()
            feature = feature.to(device)
            label = torch.tensor([label],dtype=torch.float32)
            label = label.to(device)
            
            opt.zero_grad()
            
            out = model(feature)
            
            
            loss=loss_fn(out,label)
            
            loss.backward()
            opt.step()
            running_loss += loss
        print("epoch %d loss is %f"%(epoch,running_loss))
        test2(model,test_features)

In [194]:
from sklearn.metrics import roc_auc_score
def test2(model,test_features):
    res =0
    y_test = []
    y_pred = []
    model.eval()
    for feature,label in test_features:
        feature = feature.float()
        feature = feature.to(device)
        out = model(feature)
        
        y_test.append(label)
        y_pred.append(out.item())
        if out>=0.5 and label ==1:
            res+=1
        if out<=0.5 and label ==0:
            res+=1
    print("acc is %f"%(res/len(test_features)))
    if res/len(test_features) < 0.5:
        for i in range(len(y_pred)):
            y_pred[i] = float(1-y_pred[i])
    print("AUC score is %f"%roc_auc_score(y_test,y_pred))

In [195]:
import os
modelPath='./model.pkl'

if os.path.exists(modelPath) and False:
    model = torch.load(modelPath)
    print('model exists')
else:
    model = TDGNN(in_feature=2000,hidden=256, out_feature=1)
    print("model created")

model = model.to(device)

model created


In [196]:
train(model,train_features)

epoch 0 begins


/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0 loss is 4894.352539
acc is 0.893687
AUC score is 0.899009
epoch 1 begins


/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1 loss is 4392.947266
acc is 0.895335
AUC score is 0.901031
epoch 2 begins


...

epoch 197 loss is 1769.000000
acc is 0.954475
AUC score is 0.954635
epoch 198 begins


/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 198 loss is 1769.000000
acc is 0.954475
AUC score is 0.954635
epoch 199 begins


/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 199 loss is 1769.000000
acc is 0.954475
AUC score is 0.954635


In [197]:
torch.save(model,'./model.pkl')

/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type TDGNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [32]:
torch.save(model,'./model_0.7385_0.5_mean.pkl')

/usr/local/anaconda3/envs/graph/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type TDGNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
y_pred = [0.3,0.7]
for i in range(len(y_pred)):
    y_pred[i] = 1.0-y_pred[i]
y_pred